<a href="https://colab.research.google.com/github/WillN202/NLU_CW/blob/main/NLU_Task_2_TrainedWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLU Task 2


For this task I've implemented a GRU model using TODO word embeddings. TODO WRITE ME

## Imports and Setup

In [3]:
!pip install torchmetrics
!pip install ray[tune]
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchmetrics
import os.path
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch
import string
import re
import gensim.downloader
import tempfile
import random
TRAINING_DATASET_LOCATION = "/content/drive/MyDrive/train.csv"
DEV_DATASET_LOCATION = "/content/drive/MyDrive/dev.csv"
TEST_DATASET_LOCATION = "/content/drive/MyDrive/AV_trial.csv"
WORD2VEC_EMBEDDINGS = "/content/drive/MyDrive/word2vec_embeddings_tessty.model"
EPOCHS = 4

DEVICE = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(DEVICE)
# torch.set_default_device(DEVICE)

## Data Loading and Pre Processing

In [6]:
def generic_preprocessor(sentence):
  sentence = sentence.lower()

  return sentence

class AVDataset(Dataset):
  def __init__(self, csv_file, pre_processor=None):
    self.samples = pd.read_csv(csv_file)
    self.pre_processor = pre_processor

  def __len__(self):
      return len(self.samples)

  def __getitem__(self, index):
    sample = self.samples.iloc[index]
    # sample[0] = re.sub(f"[{re.escape(string.punctuation)}]", "", sample[0])
    # sample[0] = re.sub(f"[{re.escape(string.punctuation)}]", "", sample[1])
    sample_text = f"{sample[0]} <sep> {sample[1]}"
    return (self.pre_processor(sample_text), sample[2])


In [7]:
training_samples = AVDataset(TRAINING_DATASET_LOCATION, pre_processor=generic_preprocessor)
dev_samples = AVDataset(DEV_DATASET_LOCATION, pre_processor=generic_preprocessor)
test_samples = AVDataset(TEST_DATASET_LOCATION, pre_processor=generic_preprocessor)

## Word Embedding Generation

In [8]:
def add_unk(phrase1, phrase2):
  phrase1 = ["<UNK>" if random.random() < 0.05 else word for word in phrase1.split()]
  phrase2 = ["<UNK>" if random.random() < 0.05 else word for word in phrase2.split()]

  return ["<pad>"] * random.randint(1,15) + phrase1 + ["<sep>"] + phrase2


def generate_word2vec_embeddings(size, phrases):
  return Word2Vec(sentences=phrases, workers=300, min_count=1, vector_size=size)

phrases = pd.read_csv(TRAINING_DATASET_LOCATION)
phrases = phrases.loc[:, "text_1":"text_2"].to_numpy().flatten().tolist()
phrases = [add_unk(str(phrases[i]), str(phrases[i+1])) for i in range(0, len(phrases), 2)]

embeddings_1024 = generate_word2vec_embeddings(1024, phrases)
embeddings_512 = generate_word2vec_embeddings(512, phrases)
embeddings_256 = generate_word2vec_embeddings(256, phrases)
embeddings_128 = generate_word2vec_embeddings(128, phrases)
embeddings_64 = generate_word2vec_embeddings(64, phrases)

## Model Creation

To reduce code content, the GRU models are defined from a base class. New linear layers are added between each class of model since this could not be done using parameters alone

In [9]:
class BaseGruRNN(torch.nn.Module):
  def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
    super(BaseGruRNN, self).__init__()
    self.vocab = vocab
    self.get_embedding = torch.nn.Embedding.from_pretrained(embeddings)
    self.GRU_Layer = torch.nn.GRU(embedding_size, hidden_size, batch_first=True, num_layers=rnn_layers, dropout=0.1, bidirectional=is_bidirectional)

  def forward(self, x, linear_layer):
    unk_embedding = self.vocab["<UNK>"]
    #x = [re.sub(f"[{re.escape(string.punctuation)}]", "", sentence) for sentence in x]
    x = [sentence.split() for sentence in x]
    x = [[self.vocab.get(word, unk_embedding) for word in sentence ] for sentence in x]
    # TODO -> instead of padding, use pack sequence instead. Note this may break the output from the lstm (woo)
    max_len = max([len(words) for words in x])
    x = [([self.vocab["<pad>"]] * (max_len -  len(words))) + words for words in x]
    input = torch.tensor(x)

    embeddings = self.get_embedding(input)
    GRU_int_results = self.GRU_Layer(embeddings)[1]
    GRU_values = GRU_int_results[-1] #Get last hidden state(s)
    result = linear_layer(GRU_values)
    return result

class OneLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      self.base = super(OneLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, output_size),
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class TwoLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super().__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class ThreeLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super(ThreeLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )

    def forward(self, x):
      return super().forward(x, self.linear_layer)

class FourLinearLayerGruRNN(BaseGruRNN):
    def __init__(self, embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers=1, is_bidirectional=False):
      super(FourLinearLayerGruRNN, self).__init__(embedding_size, output_size, hidden_size, embeddings, vocab, rnn_layers, is_bidirectional)
      self.linear_layer = torch.nn.Sequential(
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, hidden_size),
          torch.nn.LeakyReLU(),
          torch.nn.Dropout(p=0.1),
          torch.nn.Linear(hidden_size, output_size)
      )


    def forward(self, x):
      return super().forward(x, self.linear_layer)


## Model Training

In [10]:
def train_and_validate_loop(model, batch_size, learning_rate, epochs, tuning=False):
  device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)
  torch.set_default_device(device)
  loss_function = torch.nn.BCEWithLogitsLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  training_loader = DataLoader(training_samples, batch_size=batch_size, generator=torch.Generator(device=device))
  dev_loader = DataLoader(dev_samples, batch_size=batch_size, generator=torch.Generator(device=device))
  accuracy = torchmetrics.Accuracy(task="binary")
  f1_score = torchmetrics.F1Score(task="binary")
  mcc = torchmetrics.MatthewsCorrCoef(task="binary")
  torch.set_grad_enabled(True)

  if not tuning:
    print(f"Epochs: {epochs}")

  for epoch in range(0, epochs):
      model.train()
      running_loss = 0.0

      # Training
      for index, value in enumerate(training_loader):
          optimizer.zero_grad()
          data, labels = value
          labels = labels.reshape(-1,1)
          labels = labels.type(torch.FloatTensor)
          labels = labels.to(device)
          outputs = model(data)

          loss = loss_function(outputs, labels)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()

      # Validation to ensure the model is learning
      model.eval()
      torch.set_grad_enabled(False)
      running_loss = 0
      num_correct = 0

      pred = torch.Tensor().to(device)
      gold_standard = torch.Tensor().to(device)
      for index, value in enumerate(dev_loader):
          data, labels = value
          labels = labels.reshape(-1,1)
          labels = labels.type(torch.FloatTensor)
          labels = labels.to(device)

          outputs = model(data)
          loss = loss_function(outputs, labels)
          running_loss += loss

          normalised_outputs = torch.sigmoid(outputs)
          pred = torch.cat((pred, normalised_outputs))
          gold_standard = torch.cat((gold_standard, labels))

      torch.set_grad_enabled(True)
      batch_loss = running_loss / len(dev_loader)
      f1 = f1_score(pred, gold_standard)
      acc = accuracy(pred, gold_standard)
      m_score = mcc(pred, gold_standard)

      score = (float(acc)) + (1.5 * float(m_score))
      with tempfile.TemporaryDirectory() as temp_checkpoint_dir:
        checkpoint = None
        if (epoch + 1) % 3 == 0:
            # This saves the model to the trial directory
            torch.save(
                model.state_dict(),
                os.path.join(temp_checkpoint_dir, "model.pth")
            )
            checkpoint = train.Checkpoint.from_directory(temp_checkpoint_dir)

        if tuning:
          train.report({"score": score}, checkpoint=checkpoint)
        else:
          print(f"---------------------EPOCH {epoch+1} / {epochs}---------------------")
          print(f"Batch Loss {batch_loss}")
          print(f"Accuracy {accuracy(pred, gold_standard)}")


In [ ]:
# test_embeddings = gensim.downloader.load("word2vec-google-news-300")
# test_embeddings["<UNK>"] = np.random.rand(300)
# test_embeddings["<sep>"] = np.random.rand(300)
# test_embeddings["<pad>"] = np.random.rand(300)
# coded_embeddings = torch.FloatTensor(test_embeddings.vectors).to(DEVICE)
# vocab = test_embeddings.key_to_index
# model = FourLinearLayerGruRNN(300, 1, 300, coded_embeddings, vocab, 1, is_bidirectional=True).to(DEVICE)
# train_and_validate_loop(model,64, 0.000019238, 3)

## Hyperparameter Selection

In [11]:
def tune_training_step(config, embedding, network):
  device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "cpu"
)

  chosen_network = network[config["linear_layers"]]
  embedding = embedding[config["embedding_size"]]
  coded_embeddings = torch.FloatTensor(np.array(embedding.wv.vectors)).to(device)
  vocab = embedding.wv.key_to_index
  generated_model = chosen_network(config["embedding_size"], 1, config["embedding_size"], coded_embeddings, vocab, config["rnn_layers"], config["is_bidirectional"]).to(device)
  train_and_validate_loop(generated_model, config["batch_size"], config["lr"], config["epochs"], True)


config = {
  "lr": tune.loguniform(1e-6, 1),
  "batch_size": tune.choice([8, 16, 32, 64, 128]),
  "is_bidirectional": tune.choice([True, False]),
  "epochs": tune.choice(list(range(1,15))),
  "linear_layers": tune.choice(list(range(1,5))),
  "rnn_layers": tune.choice(list(range(1,10))),
  "embedding_size": tune.choice([64,128,256,512, 1024]),
  "hidden_size": tune.choice(list(range(128, 1025, 128)))
}


embedding = {
    64: embeddings_64,
    128: embeddings_128,
    256: embeddings_256,
    512: embeddings_512,
    1024: embeddings_1024
}

network = {
    1: OneLinearLayerGruRNN,
    2: TwoLinearLayerGruRNN,
    3: ThreeLinearLayerGruRNN,
    4: FourLinearLayerGruRNN
}

asha_scheduler = ASHAScheduler(
        metric="score",
        mode="max",
        max_t=14,
        grace_period=6,
        reduction_factor=2
    )

chosen_search_alg = HyperOptSearch(metric="score", mode="max")

training_wrapper = tune.with_resources(tune.with_parameters(tune_training_step, embedding=embedding, network=network), {"CPU": 1.6, "GPU": 0.25})
tuner = tune.Tuner(
    training_wrapper,
    tune_config=tune.TuneConfig(
        num_samples=35,
        search_alg=chosen_search_alg,
        scheduler=asha_scheduler,
    ),
    param_space=config
)

result = tuner.fit()


/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-04-24 00:49:18,547	INFO worker.py:1749 -- Started a local Ray instance.
2024-04-24 00:49:20,590	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-04-24 00:49:20,593	INFO tune.py:633 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+---------------------------------------------------------------------------+
| Configuration for experiment     tune_training_step_2024-04-24_00-49-15   |
+---------------------------------------------------------------------------+
| Search algorithm                 SearchGenerator                          |
| Scheduler                        AsyncHyperBandScheduler                  |
| Number of trials                 35                                       |
+---------------------------------------------------------------------------+

View detailed results here: /root/ray_results/tune_training_step_2024-04-24_00-49-15
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-24_00-49-15_723019_212/artifacts/2024-04-24_00-49-20/tune_training_step_2024-04-24_00-49-15/driver_artifacts`

Trial status: 1 PENDING
Current time: 2024-04-24 00:49:21. Total running time: 0s
Logical resource usage: 0/16 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:L4)
+----

(tune_training_step pid=15245) /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
(tune_training_step pid=15245)   warnings.warn("dropout option adds dropout after all but last "



Trial tune_training_step_d8a71156 started with configuration:
+------------------------------------------------------+
| Trial tune_training_step_d8a71156 config             |
+------------------------------------------------------+
| batch_size                                       128 |
| embedding_size                                   128 |
| epochs                                             1 |
| hidden_size                                      512 |
| is_bidirectional                                   1 |
| linear_layers                                      3 |
| lr                                           0.00092 |
| rnn_layers                                         5 |
+------------------------------------------------------+

Trial tune_training_step_58b81384 started with configuration:
+----------------------------------------------------+
| Trial tune_training_step_58b81384 config           |
+----------------------------------------------------+
| batch_size             

(tune_training_step pid=15435) /usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
(tune_training_step pid=15435)   warnings.warn("dropout option adds dropout after all but last "


...

Trial status: 16 TERMINATED | 3 RUNNING | 16 ERROR
Current time: 2024-04-24 03:11:08. Total running time: 2hr 21min 47s
Logical resource usage: 4.800000000000001/16 CPUs, 0.75/1 GPUs (0.0/1.0 accelerator_type:L4)
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                    status                lr     batch_size   is_bidirectional       epochs     linear_layers     rnn_layers     embedding_size     hidden_size     iter     total time (s)      score |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| tune_training_step_5e105fdc   RUNNING      0.0994704                8   True                        4                 2              5            

(tune_training_step pid=29001) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-24_00-49-15/tune_training_step_6cdb1b3f_15_batch_size=8,embedding_size=1024,epochs=14,hidden_size=384,is_bidirectional=True,linear_layers=1,lr_2024-04-24_01-39-55/checkpoint_000003)
(tune_training_step pid=38589) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-24_00-49-15/tune_training_step_eeedd63c_20_batch_size=16,embedding_size=1024,epochs=3,hidden_size=128,is_bidirectional=True,linear_layers=4,lr_2024-04-24_01-55-03/checkpoint_000000)



Trial tune_training_step_eeedd63c completed after 3 iterations at 2024-04-24 03:18:05. Total running time: 2hr 28min 44s
+----------------------------------------------------------------+
| Trial tune_training_step_eeedd63c result                       |
+----------------------------------------------------------------+
| checkpoint_dir_name                          checkpoint_000000 |
| time_this_iter_s                                    1222.73959 |
| time_total_s                                        3663.81842 |
| training_iteration                                           3 |
| score                                                  0.49802 |
+----------------------------------------------------------------+

Trial status: 17 TERMINATED | 2 RUNNING | 16 ERROR
Current time: 2024-04-24 03:18:09. Total running time: 2hr 28min 48s
Logical resource usage: 3.2/16 CPUs, 0.5/1 GPUs (0.0/1.0 accelerator_type:L4)
+---------------------------------------------------------------------------

2024-04-24 03:33:43,868	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-04-24 03:33:43,874	INFO tune.py:1021 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/tune_training_step_2024-04-24_00-49-15' in 0.0163s.
2024-04-24 03:33:43,893	ERROR tune.py:1049 -- Trials did not complete: [tune_training_step_2f0bcead, tune_training_step_86f0eda5, tune_training_step_7b11e5a4, tune_training_step_62ac763c, tune_training_step_27d15d68, tune_train


Trial tune_training_step_5e105fdc completed after 4 iterations at 2024-04-24 03:33:43. Total running time: 2hr 44min 22s
+------------------------------------------------------+
| Trial tune_training_step_5e105fdc result             |
+------------------------------------------------------+
| checkpoint_dir_name                                  |
| time_this_iter_s                             1363.02 |
| time_total_s                                 8056.74 |
| training_iteration                                 4 |
| score                                        0.50198 |
+------------------------------------------------------+

Trial status: 19 TERMINATED | 16 ERROR
Current time: 2024-04-24 03:33:43. Total running time: 2hr 44min 22s
Logical resource usage: 1.6/16 CPUs, 0.25/1 GPUs (0.0/1.0 accelerator_type:L4)
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Model Evaluation

In [12]:
best_result = result.get_best_result("score", mode="max")
with best_result.checkpoint.as_directory() as checkpoint_dir:
    state_dict = torch.load(os.path.join(checkpoint_dir, "model.pth"))

print(best_result)

Result(
  metrics={'score': 0.8920048773288727},
  path='/root/ray_results/tune_training_step_2024-04-24_00-49-15/tune_training_step_2d98e058_6_batch_size=128,embedding_size=512,epochs=12,hidden_size=128,is_bidirectional=True,linear_layers=2,lr_2024-04-24_00-50-45',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/root/ray_results/tune_training_step_2024-04-24_00-49-15/tune_training_step_2d98e058_6_batch_size=128,embedding_size=512,epochs=12,hidden_size=128,is_bidirectional=True,linear_layers=2,lr_2024-04-24_00-50-45/checkpoint_000003)
)


In [16]:
chosen_embedding = embeddings_512
coded_embeddings = torch.FloatTensor(chosen_embedding.wv.vectors).to(DEVICE)
vocab = chosen_embedding.wv.key_to_index
model = TwoLinearLayerGruRNN(512, 1, 512, coded_embeddings, vocab, 2, is_bidirectional=True).to(DEVICE)
model.load_state_dict(state_dict)


<All keys matched successfully>

In [21]:
model.eval()
torch.set_default_device(DEVICE)
torch.set_grad_enabled(False)
num_correct = 0
loader = DataLoader(test_samples, batch_size=64, generator=torch.Generator(device=DEVICE))

answers = torch.Tensor().to(DEVICE)
gold_standard = torch.Tensor().to(DEVICE)

for index, value in enumerate(loader):
    data, labels = value
    labels = labels.reshape(-1,1)
    labels = labels.type(torch.FloatTensor)
    labels = labels.to(DEVICE)

    outputs = model(data)

    answers = torch.cat((answers, outputs))
    gold_standard = torch.cat((gold_standard, labels))


torch.set_grad_enabled(True)
answers = torch.sigmoid(answers)
accuracy = torchmetrics.Accuracy(task="binary")
mcc = torchmetrics.MatthewsCorrCoef(task="binary")
f1_score = torchmetrics.F1Score(task="binary")
print(f"Accuracy {accuracy(answers, gold_standard)}")
print(f"MCC {mcc(answers, gold_standard)}")
print(f"F1 {f1_score(answers, gold_standard)}")


Accuracy 0.9399999976158142
MCC 0.8864052295684814
F1 0.936170220375061
